1. KHAI BÁO THƯ VIỆN

In [101]:
import pandas as pd
import re

2. LOAD DATA VÀ XEM DỮ LIỆU

In [102]:
data = pd.read_csv("E:\DATN\Sentiment_Analysis\data\data_source\sentimentdataset.csv")
df = pd.DataFrame(data)
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\KHANH\AppData\Local\Temp\ipykernel_3036\2617770899.py:1: SyntaxWarning: invalid escape sequence '\D'
  data = pd.read_csv("E:\DATN\Sentiment_Analysis\data\data_source\sentimentdataset.csv")


,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


3. KIỂM TRA VÀ TÌM HIỂU DỮ LIỆU 

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  732 non-null    int64  
 1   Unnamed: 0    732 non-null    int64  
 2   Text          732 non-null    object 
 3   Sentiment     732 non-null    object 
 4   Timestamp     732 non-null    object 
 5   User          732 non-null    object 
 6   Platform      732 non-null    object 
 7   Hashtags      732 non-null    object 
 8   Retweets      732 non-null    float64
 9   Likes         732 non-null    float64
 10  Country       732 non-null    object 
 11  Year          732 non-null    int64  
 12  Month         732 non-null    int64  
 13  Day           732 non-null    int64  
 14  Hour          732 non-null    int64  
dtypes: float64(2), int64(6), object(7)
memory usage: 85.9+ KB


In [104]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0.1,732.0,366.464481,211.513936,0.0,183.75,366.5,549.25,732.0
Unnamed: 0,732.0,369.740437,212.428936,0.0,185.75,370.5,553.25,736.0
Retweets,732.0,21.508197,7.061286,5.0,17.75,22.0,25.00,40.0
Likes,732.0,42.901639,14.089848,10.0,34.75,43.0,50.00,80.0
Year,732.0,2020.471311,2.802285,2010.0,2019.00,2021.0,2023.00,2023.0
Month,732.0,6.122951,3.411763,1.0,3.00,6.0,9.00,12.0
Day,732.0,15.497268,8.474553,1.0,9.00,15.0,22.00,31.0
Hour,732.0,15.521858,4.113414,0.0,13.00,16.0,19.00,23.0


In [105]:
df.isnull().sum()

Unnamed: 0.1    0
Unnamed: 0      0
Text            0
Sentiment       0
Timestamp       0
User            0
Platform        0
Hashtags        0
Retweets        0
Likes           0
Country         0
Year            0
Month           0
Day             0
Hour            0
dtype: int64

In [106]:
df.duplicated().sum()

0

In [107]:
df.shape

(732, 15)

Sau khi kiểm tra dữ liệu thì có nhưng điểm chính như sau:

    + Tổng quang: có 732 dòng và 15 cột
    + Gía trị thiếu: không có giá trị thiếu
    + Các trường dữ liệu: trong đó chỉ có 1 trường là phân loại: 'Timestamp'(theo dạn ngày),
    còn lại hậu như là theo dạng số và chuỗi.
    + Ngoại lệ: không có các ngoại lệ
    + Các dữ liệu trùng lập: không có dữ liệu trùng lập
-----------------------------------------------------------------

4. Xử lí dữ liệu

4.1 Xóa các dữ liệu lỗi

In [108]:
# Xóa cột
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [109]:
# Xóa các kí tự đặc biệt
# Hàm loại bỏ ký tự đặc biệt, emoji và giữ lại chữ cái, chữ số
def clean_text(text):
    # Sử dụng regex để loại bỏ ký tự đặc biệt và emoji
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Chỉ giữ lại chữ cái và chữ số
    text = re.sub(r'\s+', ' ', text).strip()  # Xóa khoảng trắng thừa
    return text

# Áp dụng hàm vào cột 'Text'
df['Text'] = df['Text'].apply(clean_text)

4.2. Chuẩn hóa dữ liệu

In [110]:
# cột Timestamp...
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

4.3 Data sau khi xử lí

In [111]:
df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,Traffic was terrible this morning,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,Just finished an amazing workout,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


5. Lưu dữ liệu về file csv

In [112]:
# Lưu df đã xử lí
df.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\KHANH\AppData\Local\Temp\ipykernel_3036\3901354743.py:2: SyntaxWarning: invalid escape sequence '\D'
  df.to_csv("E:\DATN\Sentiment_Analysis\data\cleaned_data\sentimentdataset.csv", index=False)
